<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/xyz_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-05-31 20:37:57--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8175 (8.0K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   7.98K  --.-KB/s    in 0s      

2022-05-31 20:37:57 (103 MB/s) - ‘utils.py’ saved [8175/8175]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import ModelCreator, get_train_val_ds, ErrorAnalyzer, get_class_weight

     |████████████████████████████████| 25.9 MB 1.2 MB/s 


In [4]:
BATCH_SIZE = 32
IMG_SIZE = (512, 512)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE)

Found 16580 files belonging to 16 classes.
Found 3508 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [9]:
xyz_creator = ModelCreator(HUB_URL, 'xyz-model')
xyz_model = xyz_creator.make_model()
callbacks = xyz_creator.get_callbacks()

In [10]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=10,
                        class_weight=class_weight)

Epoch 1/10
519/519 [==============================] - 2600s 5s/step - loss: 1205.7914 - precision: 0.5734 - recall: 0.5730 - auc: 0.4025 - accuracy: 0.5731
Epoch 2/10
519/519 [==============================] - 639s 1s/step - loss: 899.1716 - precision: 0.6524 - recall: 0.6522 - auc: 0.4909 - accuracy: 0.6524
Epoch 3/10
519/519 [==============================] - 633s 1s/step - loss: 767.2985 - precision: 0.6852 - recall: 0.6852 - auc: 0.5306 - accuracy: 0.6852
Epoch 4/10
519/519 [==============================] - 636s 1s/step - loss: 574.7043 - precision: 0.7074 - recall: 0.7074 - auc: 0.5611 - accuracy: 0.7074
Epoch 5/10
519/519 [==============================] - 635s 1s/step - loss: 415.6926 - precision: 0.7311 - recall: 0.7309 - auc: 0.5958 - accuracy: 0.7310
Epoch 6/10
519/519 [==============================] - 637s 1s/step - loss: 304.3033 - precision: 0.7480 - recall: 0.7476 - auc: 0.6251 - accuracy: 0.7478
Epoch 7/10
519/519 [==============================] - 627s 1s/step - loss:

In [11]:
xyz_model.save_weights('drive/MyDrive/checkpoints/xyz-weights.h5')

### Second day

In [ ]:
analyzer = ErrorAnalyzer(model=xyz_model, model_name='XYZ-model', ds=val_ds, classes=classes)

Making confusion matrix:


  0%|          | 0/110 [00:00<?, ?it/s]

In [ ]:
analyzer.plot_confusion_mat()

In [ ]:
analyzer.evaluate_model()

In [ ]:
xyz_model.load_weights('drive/MyDrive/checkpoints/xyz-weights.h5')

In [ ]:
history_2 = xyz_model.fit(train_ds,
                          callbacks=callbacks,
                          epochs=10,
                          class_weight=class_weight,
                          initial_epoch=5)